In [1]:
from openai import OpenAI
import math
import numpy as np
import matplotlib.pyplot as plt
import os

In [138]:
client = OpenAI()

In [139]:
def get_response(conversation):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=conversation
    )

    return completion.choices[0].message

In [140]:
dim = 1
epsilon = 0.001
domain_noise = np.random.normal(0, 1, dim)  # shift x value in each dimesion by a fixed value
domain = (-10.0, 10.0) # f(x)
steps = 30
#domain = (-30.0, 30.0) # ackley
print(domain_noise)

[0.6816302]


In [141]:
def f(x):
    dim = len(x)
    x = np.array(x) + domain_noise
    value = np.sum([xi**2 + 5*math.sin(2*xi) for xi in x])/dim

    return value

In [142]:
def ackley_nd(x):
    dim = len(x)
    x = np.array(x) + domain_noise
    total = np.sqrt(np.sum(np.square(x))/dim)
    cos_total = np.sum(np.array([math.cos(2*math.pi*xi) for xi in x]))/dim
    value = 20 + math.exp(1) - 20*math.exp(-0.2*total) - math.exp(cos_total)

    return value

In [143]:
a = 10
def rastrigin_nd(x):
    dim = len(x)
    x = np.array(x) + domain_noise
    total = np.sum(np.array([(xi**xi - a*math.cos(2*math.pi*xi)) for xi in x]))
    value = a*d - total

    return value

In [144]:
func = f

In [145]:
description_text = "You are an optimization assistant, helping me find the global minimum of a black-box function. The output must " + \
            "be only the numerical value of {domain} in the first line and explanation why you chose the value in the second line."
description_without_explanation = "You are an optimization assistant, helping me find the global minimum of a black-box function. The " + \
            "output must be only the numerical value of {domain} to evaluate the function. I will respond with the functional evaluation."
# from heni's booklet
system_description = "Now you will help me minimize a function. I have some points {degree} and the function values at those points " + \
            "f({degree}). These examples are provided below in the table. Note that the lower function values are considered better."
user_description = "Give me a new unique {degree} different from above values, which will have a function value lower than any of the " + \
            "above. Do not write code and do not stop. The output must have the format -- *x: suggested value of {degree}* in the first " + \
            "line and the explanation for the selection in the next line. Please ensure the values are different from any x value you " + \
            "have suggested before."
number_of_examples = 3
degree_string = ", ".join([f"x_{i+1}" for i in range (0, dim)]) if dim > 1 else "x"

In [146]:
def convert_example(x, f_x):
    output = ""
    for i in range(0, dim):
        output += f"{degree_string}: {x[i]}\t"

    output += f"f({degree_string}): {f_x}"
    return output

In [147]:
# reset training set
evaluations = list()
low = domain[0]
high = domain[1]
for _ in range(0, number_of_examples):
    x = np.random.uniform(low, high, dim)
    f_x = func(x)
    evaluations.append((x, f_x))
    
evaluations

[(array([-5.60245511]), 26.239028663104214),
 (array([4.23840687]), 22.189460013722158),
 (array([5.61692793]), 39.82553865703698)]

In [148]:
def parse_response(response):
    # first line has values
    res = response.split("\n")[0]
    # print(res)
    
    # sanity check non parseable response
    # print("\t" in res, ":" in res, "," in res)
    if not ("\t" in res or ":" in res or "," in res):
        print("response not parseable", response)
        return None

    response_x = res.split("\t") if "\t" in res else res
    # print("after tab removal -- ", response_x)
    #print("--------- values received ", response_x)

    response_x = response_x.split(",") if "," in response_x else response_x
    if type(response_x) == str:
        response_x = [response_x]
        
    x = list()
    #x = [float(num.strip().split(":")[1]) for num in response_x]
    for num in response_x:
        num = num.strip()
        # print(num)
        if ":" in num:
            num = num.split(":")[1]

        # print("after split", num)
        x.append(float(num))
    
    # dimensionality check for x
    if len(x) < dim:
        print("size of x smaller than dimension", response)
        return None

    return x

In [149]:
conversation = list()
x_val = list()
iterations = 30

In [150]:
output_format = "\t".join([f"x_{i+1}" for i in range(0, dim)])
#initial_prompt = description_without_explanation.format(domain = ", ".join([f"x_{(i+1)}" for i in range(0, dim)])) + "\n\ntraining_samples:\n"
system_prompt = system_description.format(degree = degree_string)

user_prompt = ""
for ev in evaluations:
    user_prompt += f"{convert_example(ev[0], ev[1])}\n"
    
user_prompt += user_description.format(degree= degree_string, \
                    definition = f"{degree_string} {'is the numerical value.' if dim == 1 else 'are the numerical values.'}")
print(system_prompt)
print(user_prompt)

# initial_prompt += "\n----- LLM Numerical Optimization Process:\n\nStep 0:\n"
# print(initial_prompt)
conversation.append({"role": "system", "content": system_prompt})
conversation.append({"role": "user", "content": user_prompt})
# print(conversation)
response = get_response(conversation)

step_number = 1
for _ in range(iterations):
    # parse response
    res = response.content.strip()
    # print(res)
    x = parse_response(res)
    conversation.append({"role": "assistant", "content": res})

    if not x:
        print("some error occurred in parsing for x value. None returned")
        break
    
    f_x = func(x)
    x_val.append(x)
    print("next evaluation", x, f_x)
    evaluations.append((x, f_x))

    if abs(evaluations[-1][1]) < epsilon:
        print(f"result reached in steps {step_number+1}")
        break

    # make the next call
    prompt = f"f(x): {f_x} \n\n Step: {step_number}\n"
    step_number += 1
    conversation.append({"role": "user", "content": prompt})
    response = get_response(conversation)

print(evaluations)

Now you will help me minimize a function. I have some points x and the function values at those points f(x). These examples are provided below in the table. Note that the lower function values are considered better.
x: -5.602455108906026	f(x): 26.239028663104214
x: 4.238406865218279	f(x): 22.189460013722158
x: 5.616927932045279	f(x): 39.82553865703698
Give me a new unique x different from above values, which will have a function value lower than any of the above. Do not write code and do not stop. The output must have the format -- *x: suggested value of x* in the first line and the explanation for the selection in the next line. Please ensure the values are different from any x value you have suggested before.
next evaluation [3.5] 21.851512995755755
next evaluation [3.8] 22.311546296372647
next evaluation [3.6] 22.126516194044108
next evaluation [3.4] 21.422471812779705
next evaluation [3.3] 20.823554151541547
next evaluation [3.2] 20.046491294465305
next evaluation [3.1] 19.09091394

KeyboardInterrupt: 

In [ ]:
# base plot
x_values = np.linspace(-10, 10, 400)
y_values = np.array([f([xi]) for xi in x_values])

plt.figure(figsize=(10, 6))
plt.plot(x_values, y_values, label='f(x)', zorder=1)

plt.scatter(current_x, f(x).numpy(), color=colors[step], zorder=2+step)

plt.title('Optimization Path on f(x)')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.legend()
plt.colorbar(plt.cm.ScalarMappable(norm=plt.Normalize(0, num_steps), cmap='viridis'), label='Step')
plt.show()